In [5]:
import pandas as pd
import json


df = pd.read_excel("validate.xlsx")
df = df.head(10)
print(df.head())

import analysisrag
import anaylysis

for i in range(len(df)):
    code = df.iloc[i]["code"]
    try: 
        vulnerabilities = analysisrag.get_vulnerabilities(code)
        df.at[i, "vulnerabilities"] = json.dumps(vulnerabilities)
    except Exception as e:
        print("error", e)
        df.at[i, "vulnerabilities"] = "nan"
    print("vulnerabilities!!!!", vulnerabilities)


                                                code  \
0  // SPDX-License-Identifier: MIT\npragma solidi...   
1  // SPDX-License-Identifier: MIT\npragma solidi...   
2  // SPDX-License-Identifier: MIT\npragma solidi...   
3  // SPDX-License-Identifier: MIT\npragma solidi...   
4  // SPDX-License-Identifier: MIT\npragma solidi...   

                                             contain  \
0  [{\n    "line": "13",\n    "token": "payable(m...   
1                                                 []   
2  [{\n    "line": "8",\n    "token": "balances[m...   
3  [\n  {\n    "line": "14",\n    "token": "Data ...   
4  [\n  {\n    "line": "10",\n    "token": "tx.or...   

                                                miss  
0                                                 []  
1  [{\n    "line": "12",\n    "token": "payable(m...  
2                                                 []  
3                                                 []  
4                                                 [

In [6]:
errors = 0
hits = 0

for i in range(len(df)):
    v = df.iloc[i]["vulnerabilities"] 
    c = df.iloc[i]["contain"] 
    m = df.iloc[i]["miss"]
    if v == "nan":
        print("vulnerabilities is nan")
        continue 
    v = json.loads(v)
    v = [i for i in v if i["severity"] in ["high", "medium"]]

    problems = [i["problem"] for i in v]
    lines = [i["line"] for i in v]
    tokens = [i["token"] for i in v]

    # print("problems: ", problems)
    # print("lines: ", lines)
    # print("tokens: ", tokens)

    for e in json.loads(m):
        if e["line"] in lines:
            errors += 1
        else: 
            hits += 1

        if e["problem"] in problems:
            errors += 1
        else: 
            hits += 1

        if e["token"] in tokens:
            errors += 1
        else: 
            hits += 1

    for e in json.loads(c):
        if e["line"] in lines:
            hits += 1
        else: 
            errors += 1

        if e["problem"] in problems:
            hits += 1
        else: 
            errors += 1

        if e["token"] in tokens:
            hits += 1
        else: 
            errors += 1

print("accuracy: ", hits/(hits+errors))       


vulnerabilities is nan
accuracy:  0.4444444444444444


In [7]:
for i in df["vulnerabilities"]:
    print(i)

[{"line": "11", "token": "require(payable(msg.sender).call{value: amount}(\"\"));", "problem": "Reentrancy Vulnerability", "severity": "high", "explanation": "This function makes an external call to transfer ETH before it updates the internal state (credit[msg.sender]). A malicious contract could exploit this vulnerability through reentrancy, allowing it to drain funds from the piggy bank by recursively calling the withdraw function.", "migration": "Use the Checks-Effects-Interactions pattern by first updating the user's credit balance before making the external call. Additionally, consider replacing `call` with `transfer` for sending ETH."}, {"line": "11", "token": "require(payable(msg.sender).call{value: amount}(\"\") );", "problem": "Improper Error Handling", "severity": "medium", "explanation": "Using `call` for transferring ETH is less safe than using `transfer` or `send`, as it does not propagate a failure exception, and if it fails for any reason, funds can be lost without rever